In [2]:
# 필요한 라이브러리 불러오기
import pandas as pd

In [15]:
# ===================== 1. 데이터 로드 =====================
# 각 데이터 파일 경로 설정
purchase_file_path = "../data/purchase_history.csv"
survey_file_path = "../data/satisfaction_survey.csv"

# CSV 파일을 데이터프레임으로 로드
df_purchase = pd.read_csv(purchase_file_path)
df_survey = pd.read_csv(survey_file_path)

In [16]:
# ===================== 2. 데이터 확인 =====================
# 데이터프레임 기본 정보 출력
print("=== 구매 이력 데이터 정보 ===")
print(df_purchase.info())  # 컬럼 정보 및 데이터 타입 확인

print("\n=== 고객 만족도 설문 데이터 정보 ===")
print(df_survey.info())  # 컬럼 정보 및 데이터 타입 확인

=== 구매 이력 데이터 정보 ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerID       99441 non-null  object 
 1   PurchaseHistory  99441 non-null  int64  
 2   TotalSpent       99441 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.3+ MB
None

=== 고객 만족도 설문 데이터 정보 ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CustomerID    99441 non-null  object 
 1   Satisfaction  98673 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.5+ MB
None


In [17]:
# 데이터의 상위 5개 행 출력
print("\n=== 구매 이력 데이터 샘플 ===")
print(df_purchase.head())

print("\n=== 고객 만족도 설문 데이터 샘플 ===")
print(df_survey.head())


=== 구매 이력 데이터 샘플 ===
                         CustomerID  PurchaseHistory  TotalSpent
0  00012a2ce6f8dcda20d059ce98491703                1      114.74
1  000161a058600d5901f007fab4c27140                1       67.41
2  0001fd6190edaaf884bcaf3d49edf079                1      195.42
3  0002414f95344307404f0ace7a26f1d5                1      179.35
4  000379cdec625522490c315e70c7a9fb                1      107.01

=== 고객 만족도 설문 데이터 샘플 ===
                         CustomerID  Satisfaction
0  00012a2ce6f8dcda20d059ce98491703           1.0
1  000161a058600d5901f007fab4c27140           4.0
2  0001fd6190edaaf884bcaf3d49edf079           5.0
3  0002414f95344307404f0ace7a26f1d5           5.0
4  000379cdec625522490c315e70c7a9fb           4.0


In [18]:
# ===================== 3. 데이터 병합 =====================
# 두 데이터셋을 CustomerID를 기준으로 병합
df_merged = pd.merge(df_purchase, df_survey, on="CustomerID")

# 병합된 데이터 확인
print("\n=== 병합된 데이터 확인 ===")
print(df_merged.head())


=== 병합된 데이터 확인 ===
                         CustomerID  PurchaseHistory  TotalSpent  Satisfaction
0  00012a2ce6f8dcda20d059ce98491703                1      114.74           1.0
1  000161a058600d5901f007fab4c27140                1       67.41           4.0
2  0001fd6190edaaf884bcaf3d49edf079                1      195.42           5.0
3  0002414f95344307404f0ace7a26f1d5                1      179.35           5.0
4  000379cdec625522490c315e70c7a9fb                1      107.01           4.0


In [20]:
# ===================== 4. 결측치 처리 =====================
# 각 컬럼별 결측치 개수 확인
print("\n=== 결측치 개수 확인 ===")
print(df_merged.isna().sum())  # 결측치 개수 출력

# # 결측치가 없는 경우 그대로 사용, 결측치가 있다면 제거 또는 적절한 값으로 대체
df_merged = df_merged.fillna(df_merged['Satisfaction'].median())

# 결측치 처리 후 데이터 확인
print("\n=== 결측치 처리 후 데이터 크기 ===")
print(df_merged.shape)  # 데이터 크기 확인


=== 결측치 개수 확인 ===
CustomerID           0
PurchaseHistory      0
TotalSpent           0
Satisfaction       768
dtype: int64

=== 결측치 처리 후 데이터 크기 ===
(99441, 4)


In [21]:
# ===================== 5. 고객 세분화 (구매 이력 기반) =====================
# 구매 횟수(PurchaseHistory)를 기준으로 고객을 세 그룹으로 나누기
# 구매 횟수가 10 이하: 'Low'
# 구매 횟수가 11~30: 'Medium'
# 구매 횟수가 31 이상: 'High'
def categorize_purchase_history(purchase_count):
    if purchase_count<=10:
        return 'Low'
    elif purchase_count<=30:
        return 'Medium'
    else:
        return 'High'

df_merged["PurchaseCategory"] = df_merged["PurchaseHistory"].apply(categorize_purchase_history)

# 고객 세분화 결과 확인
print("\n=== 구매 횟수 기반 고객 세분화 결과 ===")
print(df_merged["PurchaseCategory"].value_counts())  # 각 그룹별 고객 수 확인


=== 구매 횟수 기반 고객 세분화 결과 ===
PurchaseCategory
Low    99441
Name: count, dtype: int64


In [22]:
# ===================== 6. 고객 만족도 기반 세분화 =====================
# 만족도 점수(Satisfaction)를 기준으로 고객을 세 그룹으로 나누기
# 만족도 1~3: 'Low'
# 만족도 4~7: 'Medium'
# 만족도 8~10: 'High'
def categorize_satisfaction(satisfaction_score):
    if 1<=satisfaction_score<=3:
        return 'Low'
    elif 4<=satisfaction_score<=7:
        return 'Medium'
    else:
        return 'High'

df_merged["SatisfactionCategory"] = df_merged["Satisfaction"].apply(categorize_satisfaction)

# 고객 만족도 세분화 결과 확인
print("\n=== 고객 만족도 기반 세분화 결과 ===")
print(df_merged["SatisfactionCategory"].value_counts())  # 각 그룹별 고객 수 확인


=== 고객 만족도 기반 세분화 결과 ===
SatisfactionCategory
Medium    76796
Low       22619
High         26
Name: count, dtype: int64


In [24]:
# ===================== 7. 각 그룹별 평균 구매 금액 분석 =====================
# 구매 카테고리별 평균 총 지출 금액 분석
df_purchase_analysis = df_merged.groupby("PurchaseCategory")["TotalSpent"].mean().reset_index()
df_purchase_analysis.rename(columns={"TotalSpent": "AvgSpent"}, inplace=True)

# 구매 카테고리별 평균 지출 금액 출력
print("\n=== 구매 카테고리별 평균 지출 금액 분석 ===")
print(df_purchase_analysis)

# 만족도 카테고리별 평균 총 지출 금액 분석
df_satisfaction_analysis = df_merged.groupby("SatisfactionCategory")["TotalSpent"].mean().reset_index()
df_satisfaction_analysis.rename(columns={"TotalSpent": "AvgSpent"}, inplace=True)

# 만족도 카테고리별 평균 지출 금액 출력
print("\n=== 만족도 카테고리별 평균 지출 금액 분석 ===")
print(df_satisfaction_analysis)



=== 구매 카테고리별 평균 지출 금액 분석 ===
  PurchaseCategory    AvgSpent
0              Low  160.988648

=== 만족도 카테고리별 평균 지출 금액 분석 ===
  SatisfactionCategory    AvgSpent
0                 High  139.740385
1                  Low  176.667751
2               Medium  156.377819


In [25]:
# ===================== 8. 구매 카테고리와 만족도 카테고리 간의 관계 분석 =====================
# 구매 카테고리별 고객 만족도 분포 확인
df_category_relation = df_merged.groupby(["PurchaseCategory", "SatisfactionCategory"])["CustomerID"].count().reset_index()
df_category_relation.rename(columns={"CustomerID": "CustomerCount"}, inplace=True)

# 관계 분석 결과 출력
print("\n=== 구매 카테고리와 만족도 카테고리 간의 관계 분석 ===")
print(df_category_relation)


=== 구매 카테고리와 만족도 카테고리 간의 관계 분석 ===
  PurchaseCategory SatisfactionCategory  CustomerCount
0              Low                 High             26
1              Low                  Low          22619
2              Low               Medium          76796


In [26]:
# ===================== 9. 데이터 저장 =====================
# 분석된 데이터를 CSV 파일로 저장
df_merged.to_csv("../data/segmented_customer_data.csv", index=False)
df_purchase_analysis.to_csv("../data/purchase_category_analysis.csv", index=False)
df_satisfaction_analysis.to_csv("../data/satisfaction_category_analysis.csv", index=False)
df_category_relation.to_csv("../data/category_relation_analysis.csv", index=False)

print("\n=== 분석된 데이터 저장 완료 ===")



=== 분석된 데이터 저장 완료 ===
